In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
xls = pd.ExcelFile('RLGFactors.xlsx')
FACTORS = []

for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name = sheet, header = [0,1])
    df.set_index(('Unnamed: 0_level_0','Dates'), inplace = True)
    df = df.stack(level = [0, 1], future_stack = True).reset_index()
    df.columns = ['Date', 'Ticker','Variable','Value']

    FACTORS.append(df)

FACTORS = pd.concat(FACTORS, axis = 0)

FACTORS['Ticker'] = FACTORS['Ticker'].apply(lambda x: x.split(' ')[0])
FACTORS['Date'] = pd.to_datetime(FACTORS['Date'])
FACTORS.head()

,Date,Ticker,Variable,Value
0,2010-12-31,AAL,PE_RATIO,NaN
1,2010-12-31,AAL,IS_EPS,-1.410
2,2010-12-31,AAL,BEST_ANALYST_RATING,3.929
3,2010-12-31,AAL,BEST_SALES,NaN
4,2010-12-31,AAL,TOT_DEBT_TO_TOT_EQY,NaN


In [3]:
FACTORS = FACTORS[~FACTORS['Variable'].isin([
    'PE_RATIO.1', 'IS_EPS.1', 'BEST_ANALYST_RATING.1', 'BEST_SALES.1',
    'TOT_DEBT_TO_TOT_EQY.1', 'EBIT.1', 'RETURN_COM_EQY.1',
    'RETURN_ON_ASSET.1', 'RETURN_ON_INV_CAPITAL.1', 'TOTAL_EQUITY.1',
    'EQY_SH_OUT.1'
])]

In [4]:
factor_mapping = {
    'PE_RATIO':'PE', 
    'IS_EPS':'EPS', 
    'BEST_ANALYST_RATING':'RATING', 
    'BEST_SALES':'REVENUE',
    'TOT_DEBT_TO_TOT_EQY':'DE', 
    'EBIT':'EBIT', 
    'RETURN_COM_EQY':'ROE', 
    'RETURN_ON_ASSET':'ROA',
    'RETURN_ON_INV_CAPITAL':'ROIC', 
    'TOTAL_EQUITY':'EQUITY', 
    'EQY_SH_OUT':'SHARESOUT'
}

FACTORS['Variable'] = FACTORS['Variable'].map(factor_mapping)

In [5]:
FACTORS.to_csv('Factors.csv', index = False)

In [6]:
PRICES = pd.read_excel('RLGPrices.xlsx')
PRICES = pd.melt(PRICES, id_vars = ['Dates'], var_name = 'Ticker', value_name = 'Price')
PRICES.head()

,Dates,Ticker,Price
0,2010-06-01,LEN UN Equity,16.3006
1,2010-06-02,LEN UN Equity,16.5202
2,2010-06-03,LEN UN Equity,16.2507
3,2010-06-04,LEN UN Equity,15.4422
4,2010-06-07,LEN UN Equity,14.7334


In [ ]:
def SortingByVar(df, var, n):
    df = df[df['Variable'] == var].dropna()
    df[var] = df.groupby('Date')['Value'].transform(lambda x: pd.qcut(x, q = n, labels = False, duplicates = 'drop') + 1)
    return df[['Date','Ticker',var]]

In [ ]:
PORTFOLIOS = SortingByVar(FACTORS, 'PE', n = 10)

factors = ['RATING','']

for factor in :
    print(factor)
    df = SortingByVar(FACTORS, factor, n = 10)
    PORTFOLIOS = pd.merge(PORTFOLIOS, df, on = ['Date', 'Ticker'], how = 'left')

PORTFOLIOS.head()

EPS
RATING
REVENUE
DE
EBIT
ROE
ROA
ROIC
EQUITY
SHARESOUT


,Date,Ticker,PE,EPS,RATING,REVENUE,DE,EBIT,ROE,ROA,ROIC,EQUITY,SHARESOUT
0,2010-12-31,AAON,7,3,1.0,NaN,1.0,2.0,6.0,8.0,8.0,2.0,3.0
1,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
2,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
3,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
4,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0


In [60]:
PORTFOLIOS

,Date,Ticker,PE,EPS,RATING,REVENUE,DE,EBIT,ROE,ROA,ROIC,EQUITY,SHARESOUT
0,2010-12-31,AAON,7,3,1.0,NaN,1.0,2.0,6.0,8.0,8.0,2.0,3.0
1,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
2,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
3,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
4,2010-12-31,AAPL,3,5,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571302,2023-11-30,HAS,7,1,9.0,NaN,9.0,1.0,1.0,1.0,1.0,3.0,5.0
5571303,2023-11-30,PRDO,2,6,10.0,NaN,1.0,3.0,8.0,10.0,9.0,2.0,3.0
5571304,2023-11-30,RMBS,8,6,9.0,NaN,1.0,4.0,9.0,10.0,10.0,3.0,4.0
5571305,2023-11-30,MRVL,10,2,10.0,NaN,2.0,1.0,2.0,2.0,2.0,9.0,10.0


In [ ]:
for i in factor_mapping.values:
    SortingByVar(df)

In [64]:
variable = 'REVENUE'
# date = '2010-12-31'

df = FACTORS[FACTORS['Variable'] == variable]
# df['Portfolio'] = df.groupby('Date')['Value'].transform(lambda x: pd.qcut(x, q=4, labels=False) + 1)
df['Value'].isna().value_counts()

Value
True     10924
False     3286
Name: count, dtype: int64

In [ ]:
variable = 'DE'
date = '2010-12-31'

df = FACTORS[(FACTORS['Date'] == date) & (FACTORS['Variable'] == variable)]
df['PortfolioPE'] = pd.qcut(df['Value'], q = 4, labels = False) + 1
df

C:\Users\paulo\AppData\Local\Temp\ipykernel_12936\3171672250.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PortfolioPE'] = pd.qcut(df['Value'], q = 4, labels = False) + 1


,Date,Ticker,Variable,Value,PortfolioPE
4,2010-12-31,AAL,DE,NaN,NaN
15,2010-12-31,AAON,DE,0.0000,1.0
26,2010-12-31,AAPL,DE,0.0000,1.0
37,2010-12-31,ABNB,DE,NaN,NaN
48,2010-12-31,ADBE,DE,29.9106,2.0
...,...,...,...,...,...
3568,2010-12-31,ESI,DE,NaN,NaN
3579,2010-12-31,SYF,DE,718.4394,4.0
3590,2010-12-31,POST,DE,54.6804,3.0
3601,2010-12-31,WBC,DE,24.6739,2.0
